In [1]:
import pandas as pd
import numpy as np

from scipy.stats import chi2_contingency

### Import Data

In [2]:
fact = pd.read_excel(open('data DAE.xlsx', 'rb'), sheet_name='fact') 
d_scheme = pd.read_excel(open('data DAE.xlsx', 'rb'), sheet_name='d_scheme') 
d_voucher = pd.read_excel(open('data DAE.xlsx', 'rb'), sheet_name='d_voucher') 
d_merchant = pd.read_excel(open('data DAE.xlsx', 'rb'), sheet_name='d_merchant') 
d_user = pd.read_excel(open('data DAE.xlsx', 'rb'), sheet_name='d_user') 

# Make all headers lowercase
for df in [fact, d_scheme, d_voucher, d_merchant, d_user]:
    df.columns = map(str.lower, df.columns)

d_scheme['discount_amount'] = d_scheme['discount_amount'].astype('float')
d_voucher['total_stock'] = d_voucher['total_stock'].astype('float')

### Claim & Redemption Rate

In [3]:
claimed = fact.loc[fact['action'] == 'Claimed'][['voucher_code', 'action']].groupby('voucher_code').count().reset_index()
claimed = claimed.rename(columns={'action': 'claimed'})

redeemed = fact.loc[fact['action'] == 'Redeemed'][['voucher_code', 'action']].groupby('voucher_code').count().reset_index()
redeemed = redeemed.rename(columns={'action': 'redeemed'})

voucher_claim_redeem = d_voucher.merge(claimed, how='left', on='voucher_code')
voucher_claim_redeem = voucher_claim_redeem.merge(redeemed, how='left', on='voucher_code')
voucher_claim_redeem = voucher_claim_redeem.fillna(0)
voucher_claim_redeem['claim_rate'] = voucher_claim_redeem['claimed'] / voucher_claim_redeem['total_stock']
voucher_claim_redeem['redeem_rate'] = voucher_claim_redeem['redeemed'] / voucher_claim_redeem['total_stock']

voucher_claim_redeem = voucher_claim_redeem.merge(d_scheme[['voucher_code', 'discount_type']], how='left', on='voucher_code')

voucher_claim_redeem = voucher_claim_redeem.sort_values('redeem_rate', ascending=False)
voucher_claim_redeem.to_csv('charts/voucher_claim_redeem.csv', index=False)

In [4]:
voucher_claim_redeem.groupby('discount_type').describe()

total_stock                \
                                             count          mean   
discount_type                                                      
Buy 1 get 1                                    3.0    366.666667   
Buy 2 get 1                                    6.0  13824.666667   
Cash Voucher                                  67.0  73264.253731   
Combo Bundle                                   7.0    457.000000   
Discount Fixed Amount (For SKU)               57.0   1598.350877   
Discount Fixed Amount (For Total Bill)        52.0   4233.961538   
Discount Percentage (For SKU)                 71.0   1947.239437   
Discount Percentage (For Total Bill)          13.0   2265.538462   
Free SKU                                      37.0   6636.378378   
Others                                         2.0  14085.500000   

                                                                         \
                                                  std     min       25%   
discount_type                                                             
Buy 1 get 1                                288.675135   200.0    200.00   
Buy 2 get 1                               6948.989183  5000.0   8461.00   
Cash Voucher                            208961.674481     3.0    183.00   
Combo Bundle                               275.698507   200.0    250.00   
Discount Fixed Amount (For SKU)           3281.426826   100.0    500.00   
Discount Fixed Amount (For Total Bill)    7355.824394    20.0    382.50   
Discount Percentage (For SKU)             3909.600295    50.0    400.00   
Discount Percentage (For Total Bill)      3075.025897   100.0    200.00   
Free SKU                                 11970.426348   100.0    500.00   
Others                                   10827.926140  6429.0  10257.25   

                                                                      claimed  \
                                            50%        75%        max   count   
discount_type                                                                   
Buy 1 get 1                               200.0     450.00      700.0     3.0   
Buy 2 get 1                             15000.0   20000.00    20000.0     6.0   
Cash Voucher                              576.0  100151.50  1000569.0    67.0   
Combo Bundle                              500.0     500.00      999.0     7.0   
Discount Fixed Amount (For SKU)           500.0    1000.00    20000.0    57.0   
Discount Fixed Amount (For Total Bill)    950.0    2625.00    30000.0    52.0   
Discount Percentage (For SKU)             500.0    1260.00    20000.0    71.0   
Discount Percentage (For Total Bill)      500.0    4000.00     9960.0    13.0   
Free SKU                                 1000.0    5272.00    50000.0    37.0   
Others                                  14085.5   17913.75    21742.0     2.0   

                                                     ... claim_rate            \
                                               mean  ...        75%       max   
discount_type                                        ...                        
Buy 1 get 1                              104.666667  ...   0.237500  0.420000   
Buy 2 get 1                             3647.666667  ...   0.312500  0.734800   
Cash Voucher                             570.268657  ...   0.078743  1.000000   
Combo Bundle                               7.571429  ...   0.038000  0.075000   
Discount Fixed Amount (For SKU)          120.350877  ...   0.098000  0.618000   
Discount Fixed Amount (For Total Bill)  1828.557692  ...   0.573813  2.060000   
Discount Percentage (For SKU)            376.422535  ...   0.185502  0.980000   
Discount Percentage (For Total Bill)     356.692308  ...   0.370667  0.965000   
Free SKU                                 420.945946  ...   0.200000  0.558333   
Others                                     0.000000  ...   0.000000  0.000000   

                                       redeem_rate                    

### Merchant Category

In [5]:
c1 = d_voucher[['voucher_code', 'total_stock']].drop_duplicates()
c2 = d_merchant[['merchant_code', 'category']].drop_duplicates()
c1['key'] = 0
c2['key'] = 0
merc_base = c1.merge(c2, how='outer', on='key')

# ** Some vouchers are not claimed, but yet still have redemptions.
# ** The workaround below is to make sure there's a voucher for every merchant.

# *** Some merchants are tagged to multiple categories and sub-categories.
# *** This creates duplicates when trying to count voucher claims/redemptions for categories and sub-categories.
# *** For simplicity's sake, I have applied a quick method to attribute each voucher claim/redemption to all categories (and sub) under the same merchant, evenly.

claimed_merc =  d_voucher[['voucher_code']].merge(fact.loc[fact['action'] == 'Claimed'][['voucher_code', 'merchant_code', 'action']].groupby(['voucher_code', 'merchant_code']).count().reset_index(), how='outer', on=['voucher_code'])
claimed_merc = claimed_merc.rename(columns={'action': 'claimed'})
claimed_merc = claimed_merc.fillna(0)

redeemed_merc = d_voucher[['voucher_code']].merge(fact.loc[fact['action'] == 'Redeemed'][['voucher_code', 'merchant_code', 'action']].groupby(['voucher_code', 'merchant_code']).count().reset_index(), how='outer', on=['voucher_code'])
redeemed_merc = redeemed_merc.rename(columns={'action': 'redeemed'})

merc_agg = merc_base.merge(claimed_merc, how='left', on=['voucher_code', 'merchant_code'])
merc_agg = merc_agg.merge(redeemed_merc, how='left', on=['voucher_code', 'merchant_code'])
merc_agg = merc_agg.fillna(0)
merc_agg['total'] = merc_agg['claimed'] + merc_agg['redeemed']
merc_agg = merc_agg[merc_agg['total'] > 0]

# distribute voucher claimed & redeem evenly across categories for the same merchant
merc_agg['claimed'] = merc_agg.apply(lambda x: ( x.claimed / len(merc_agg[(merc_agg['voucher_code'] == x['voucher_code']) & (merc_agg['merchant_code'] == x['merchant_code'])] )) if True else x, axis=1)
merc_agg['redeemed'] = merc_agg.apply(lambda x: ( x.redeemed / len(merc_agg[(merc_agg['voucher_code'] == x['voucher_code']) & (merc_agg['merchant_code'] == x['merchant_code'])] )) if True else x, axis=1)
merc_agg['claim_rate'] = merc_agg['claimed'] / merc_agg['total_stock']
merc_agg['redeem_rate'] = merc_agg['redeemed'] / merc_agg['total_stock']

In [6]:
# Calculate final stats for categories

voucher_cat_stats = merc_agg[['voucher_code', 'category', 'merchant_code', 'claimed', 'redeemed', 'total_stock']].groupby(['voucher_code', 'category']).sum(numeric_only=True).reset_index()
cat_total = merc_agg[['voucher_code', 'category', 'claimed', 'redeemed']].groupby(['voucher_code', 'category']).sum(numeric_only=True).reset_index()
cat_total.to_csv('charts/cat_total.csv', index=False, encoding="utf-8")

## Merchant sub_category

In [7]:
c1 = d_voucher[['voucher_code', 'total_stock']].drop_duplicates()
c2 = d_merchant[['merchant_code', 'category', 'sub_category']].drop_duplicates()
c1['key'] = 0
c2['key'] = 0
merc_base = c1.merge(c2, how='outer', on='key')

claimed_merc =  d_voucher[['voucher_code']].merge(fact.loc[fact['action'] == 'Claimed'][['voucher_code', 'merchant_code', 'action']].groupby(['voucher_code', 'merchant_code']).count().reset_index(), how='outer', on=['voucher_code'])
claimed_merc = claimed_merc.rename(columns={'action': 'claimed'})
claimed_merc = claimed_merc.fillna(0)

redeemed_merc = d_voucher[['voucher_code']].merge(fact.loc[fact['action'] == 'Redeemed'][['voucher_code', 'merchant_code', 'action']].groupby(['voucher_code', 'merchant_code']).count().reset_index(), how='outer', on=['voucher_code'])
redeemed_merc = redeemed_merc.rename(columns={'action': 'redeemed'})

merc_agg = merc_base.merge(claimed_merc, how='left', on=['voucher_code', 'merchant_code'])
merc_agg = merc_agg.merge(redeemed_merc, how='left', on=['voucher_code', 'merchant_code'])
merc_agg = merc_agg.fillna(0)
merc_agg['total'] = merc_agg['claimed'] + merc_agg['redeemed']
merc_agg = merc_agg[merc_agg['total'] > 0]

# distribute voucher claimed & redeem evenly across categories for the same merchant
merc_agg['claimed'] = merc_agg.apply(lambda x: ( x.claimed / len(merc_agg[(merc_agg['voucher_code'] == x['voucher_code']) & (merc_agg['merchant_code'] == x['merchant_code'])] )) if True else x, axis=1)
merc_agg['redeemed'] = merc_agg.apply(lambda x: ( x.redeemed / len(merc_agg[(merc_agg['voucher_code'] == x['voucher_code']) & (merc_agg['merchant_code'] == x['merchant_code'])] )) if True else x, axis=1)
merc_agg['claim_rate'] = merc_agg['claimed'] / merc_agg['total_stock']
merc_agg['redeem_rate'] = merc_agg['redeemed'] / merc_agg['total_stock']

# Calculate final stats for sub-categories

voucher_subcat_stats = merc_agg[['voucher_code', 'category', 'merchant_code', 'claimed', 'redeemed', 'total_stock']].groupby(['voucher_code', 'category']).sum(numeric_only=True).reset_index()
subcat_total = merc_agg[['voucher_code', 'category', 'sub_category', 'claimed', 'redeemed']].groupby(['voucher_code', 'category', 'sub_category']).sum(numeric_only=True).reset_index()
subcat_total.to_csv('charts/subcat_total.csv', index=False, encoding="utf-8")

### Timeseries/Daily

In [8]:
daily_claim = fact[fact['action'] == 'Claimed'][['calendar_dim_id', 'action']]
daily_redeem = fact[fact['action'] == 'Redeemed'][['calendar_dim_id', 'action']]
daily_claim['date'] = pd.to_datetime(daily_claim['calendar_dim_id'])
daily_redeem['date'] = pd.to_datetime(daily_redeem['calendar_dim_id'])

daily_claim.groupby('date').count().reset_index().to_csv('charts/daily_claim.csv', index=False)
daily_redeem.groupby('date').count().reset_index().to_csv('charts/daily_redeem.csv', index=False)

### Claim rate by voucher release date

In [9]:
voucher_claim_redeem['display_date_from_dt'] = voucher_claim_redeem['display_date_from'].dt.floor('D')
voucher_claim_redeem.groupby('display_date_from_dt').sum().reset_index().to_csv('charts/claim_rate_release_date.csv', index=False)

C:\Users\Jason\AppData\Local\Temp\ipykernel_30212\1458008829.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  voucher_claim_redeem.groupby('display_date_from_dt').sum().reset_index().to_csv('charts/claim_rate_release_date.csv', index=False)


### User Profile

In [10]:
print(d_user.groupby('gender').describe())
print(d_user.groupby('province_name').describe())
print(d_user.groupby('age').describe())

            user_id                      province_name                      \
              count unique      top freq         count unique    top  freq   
gender                                                                       
F             38740  38740    76843    1         38740     11    HCM  3642   
M             21706  21706   cb2f6f    1         21706     11  Hànoi  2040   
unavailable      23     23  133ab84    1            23     11    HCM     5   

               age                             
             count unique           top  freq  
gender                                         
F            38740     22         45_54  4388  
M            21706     22         25_34  2504  
unavailable     23      9  UNIDENTIFIED     4  
              gender                  user_id                        age  \
               count unique top  freq   count unique      top freq count   
province_name                                                              
HCM            

### User Preference using Cramer's V function

In [11]:
user_pref = fact.merge(d_merchant, how='left', on='merchant_code')
user_pref = user_pref.merge(d_user, how='left', on='user_id')
user_pref = user_pref.merge(d_scheme, how='left', on='voucher_code')

In [12]:
# Cramer's V function
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(k - 1, r - 1))))

user_char = ['gender', 'province_name', 'age']
cat_char = ['category', 'sub_category', 'discount_type']

for u in user_char:
    for c in cat_char:
        contingency_table = pd.crosstab(user_pref[u], user_pref[c])
        cramers_v_value = cramers_v(contingency_table)
        print('======== ', u, ' --- ', c, '========')
        print(f"Cramér's V: {cramers_v_value}")

========  gender  ---  category ========
Cramér's V: category
Bán lẻ FMCG                       0.310882
Dịch vụ tài chính                27.543560
Giáo dục                          4.123167
Khác                              4.004351
Mẹ & bé                           0.439391
Nhà cửa - Đời sống                1.196204
Sức khỏe - Làm đẹp                0.412438
Thương mại điện tử                0.310880
Thời trang và Phong cách sống     0.361437
Tiện ích - Nhà mạng               0.439633
Vận tải - Giao vận                0.393530
Ẩm thực                           0.203846
dtype: float64
========  gender  ---  sub_category ========
Cramér's V: sub_category
Bảo hiểm                      33.378845
Bệnh viện - Phòng khám         5.068174
Cửa hàng kem/bánh/kẹo          1.887061
Cửa hàng tiện lợi              0.532797
Dịch vụ giao hàng              0.506679
Dịch vụ hỗ trợ gia đình        1.710050
Gia dụng                       3.578588
Hoa & Quà lưu niệm             4.881806
Hàng không       